In [1]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import classification_report
# data = pd.read_csv('cleaned_data_new.csv') 


In [2]:
data.shape

(24922, 33)

In [3]:

# # Identify numerical and categorical features
# numeric_features = data.select_dtypes(include=['int64', 'float64']).columns.to_list()
# numeric_features.remove('patient_nbr')  # Remove patient identifier from features
# categorical_features = data.select_dtypes(include=['object']).columns.to_list()

# # Define preprocessing for numerical columns 
# numeric_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='median')),
#     ('scaler', StandardScaler())])

# # Define preprocessing for categorical columns (encode them)
# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# # Combine preprocessing steps
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)])

# # Define the model
# rf_model = Pipeline(steps=[('preprocessor', preprocessor),
#                            ('classifier', RandomForestClassifier(random_state=42))])

# # Separate target variable and predictors
# X = data.drop(['readmitted', 'patient_nbr'], axis=1)  # Exclude 'readmitted' and 'patient_nbr' from features
# y = data['readmitted']

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# # Train the Random Forest model
# rf_model.fit(X_train, y_train)

# # Predict on the test data
# y_pred = rf_model.predict(X_test)

# # Evaluate the model
# report = classification_report(y_test, y_pred, output_dict=True)
# report_df = pd.DataFrame(report).transpose()
# report_df


,precision,recall,f1-score,support
False,0.901672,1.000000,0.948294,6740.000000
True,1.000000,0.002714,0.005413,737.000000
accuracy,0.901699,0.901699,0.901699,0.901699
macro avg,0.950836,0.501357,0.476853,7477.000000
weighted avg,0.911364,0.901699,0.855355,7477.000000


In [6]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, precision_recall_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

# Load your data
data = pd.read_csv('cleaned_data_new.csv') 

# Identify numerical and categorical features
numeric_features = data.select_dtypes(include=['int64', 'float64']).columns.to_list()
numeric_features.remove('patient_nbr')  # Assuming 'patient_nbr' is not a feature for modeling
categorical_features = data.select_dtypes(include=['object']).columns.to_list()

# Define preprocessing for numerical columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# Define preprocessing for categorical columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Define the Random Forest model pipeline
rf_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced_subsample'))])

# Split the data
X = data.drop(['readmitted', 'patient_nbr'], axis=1)  # Assuming these are not features
y = data['readmitted']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict probabilities
y_scores = rf_model.predict_proba(X_test)[:, 1]  # Get the probability scores for the positive class

# Determine the best threshold for recall >= 0.50 and precision >= 0.20
precision, recall, thresholds = precision_recall_curve(y_test, y_scores)
filtered_indices = np.where((recall[:-1] >= 0.50) & (precision[:-1] >= 0.20))

if filtered_indices[0].size > 0:
    threshold_best = thresholds[filtered_indices[0][0]]  # Use the first threshold that meets the criteria
    print(f"Selected threshold: {threshold_best}")
else:
    print("No threshold satisfies the desired recall and precision simultaneously.")
    # Use a default or calculate alternative threshold
    combined_scores = recall[:-1] + precision[:-1]
    best_index = np.argmax(combined_scores)
    threshold_best = thresholds[best_index]
    print(f"Using best combined score threshold: {threshold_best} with recall {recall[best_index]} and precision {precision[best_index]}.")

# Apply the new threshold to classify instances
y_pred_adjusted = (y_scores >= threshold_best).astype(int)

# Evaluation
new_report = classification_report(y_test, y_pred_adjusted, output_dict=True)
new_report_df = pd.DataFrame(new_report).transpose()
print(new_report_df)

No threshold satisfies the desired recall and precision simultaneously.
Using best combined score threshold: 0.0 with recall 1.0 and precision 0.09856894476394276.
              precision    recall  f1-score      support
False          0.000000  0.000000  0.000000  6740.000000
True           0.098569  1.000000  0.179450   737.000000
accuracy       0.098569  0.098569  0.098569     0.098569
macro avg      0.049284  0.500000  0.089725  7477.000000
weighted avg   0.009716  0.098569  0.017688  7477.000000


C:\Users\bhuva\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bhuva\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bhuva\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classificati